# Processing COVID AMP dataset for analysis and visualization

## Import

In [1]:
import pandas as pd
import numpy as np
from pyairtable import Table

from datetime import datetime
from datetime import date
import matplotlib
from matplotlib import font_manager
from matplotlib.dates import date2num
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as md
import seaborn as sns
import time
import string
import warnings
warnings.filterwarnings('ignore')

In [2]:
base_key = 'appoXaOlIgpiHK3I2'
table_name = 'Policy Database'

# AIRTABLE API KEY -- DON'T COMMIT INTO GIT!
api_key = 'MY_AIRTABLE_KEY'

In [89]:
table = Table(api_key, base_key, table_name)

records = table.all()

In [154]:
full = pd.DataFrame.from_records((r['fields'] for r in records))

## Cleaning

In [155]:
# remove non-policy guidance
full = full.loc[full['Policy/law type'] != 'Non-policy guidance']

In [156]:
# drop rows missing key information
full = full.dropna(subset=['Unique ID',
                           'Policy category', 
                           'Policy description', 
                           'Authorizing country ISO'])

In [157]:
# drop down to a select number of relevant columns
cols = ['Unique ID','Issued date','Effective start date','Anticipated end date','Actual end date','Policy category','Policy subcategory', 'Policy target','Policy relaxing or restricting','Authorizing country ISO','Auth state names (lookup)']
full = full[cols]

In [158]:
# convert to datetime for indexing
full['Issued date'] = pd.to_datetime(full['Issued date'], format='%Y/%m/%d')
full['Effective start date'] = pd.to_datetime(full['Effective start date'], format='%Y/%m/%d')
full['Anticipated end date'] = pd.to_datetime(full['Anticipated end date'], format='%Y/%m/%d')
full['Actual end date'] = pd.to_datetime(full['Actual end date'], format='%Y/%m/%d')

# extract year and month for ease of analysis
full['Start year'] = full['Effective start date'].dt.strftime('%Y')
full['Start month'] = full['Effective start date'].dt.strftime('%B')

In [159]:
# rename geo columns 
full = full.rename(columns={'Auth state names (lookup)': 'State', 'Authorizing country ISO':'Country'})

In [160]:
# remove brackets from state and country name
full['State'] = full['State'].astype(str)
full['State'] = full['State'].str.extract("\'(.*)\'")

full['Country'] = full['Country'].astype(str)
full['Country'] = full['Country'].str.extract("\'(.*)\'")

In [163]:
full.head()

,Unique ID,Issued date,Effective start date,Anticipated end date,Actual end date,Policy category,Policy subcategory,Policy target,Policy relaxing or restricting,Country,State,Start year,Start month
0,67754,2021-01-30,2021-01-30,2021-02-28,2021-02-11,Social distancing,Adaptation and mitigation measures,[Businesses/private sector],Relaxing,USA,Nebraska,2021,January
1,55202,2020-08-24,2020-08-24,NaT,NaT,Social distancing,Event delays or cancellations,[General population (inclusive)],Restricting,USA,California,2020,August
2,127069,2020-04-13,2020-04-13,NaT,NaT,Enabling and relief measures,Hazard pay,[Public sector/public buildings/government emp...,Other,USA,Puerto Rico,2020,April
3,34911,2020-07-12,2020-07-12,2020-07-17,2020-07-17,Social distancing,Private sector closures,[Restaurants/food service establishments],Relaxing,USA,New York,2020,July
4,18897,2021-01-08,2021-01-09,2021-02-07,NaT,Social distancing,Adaptation and mitigation measures,[Criminal justice system/prisons/jails/incarce...,Restricting,USA,Georgia,2021,January


In [165]:
full.to_csv('processed/airtable_cleaned_033123.csv')

## Analysis

In [166]:
clean = full.copy()